In [1]:
import json

import pandas as pd
import torch
from tqdm import tqdm


def invert_scaling_torch(
    scaled_series: torch.Tensor, uid: str, stats_dict: dict, device="cpu"
) -> torch.Tensor:
    scaler_info = stats_dict.get(uid)
    if not scaler_info:
        raise ValueError(f"No scaler info found for unique_id '{uid}'")

    scaled_series = scaled_series.to(device)

    if scaler_info["type"] == "minmax":
        min_val = torch.tensor(scaler_info["min"], dtype=torch.float32, device=device)
        max_val = torch.tensor(scaler_info["max"], dtype=torch.float32, device=device)
        return scaled_series * (max_val - min_val + 1e-8) + min_val

    elif scaler_info["type"] == "standard":
        mean = torch.tensor(scaler_info["mean"], dtype=torch.float32, device=device)
        std = torch.tensor(scaler_info["std"], dtype=torch.float32, device=device)
        return scaled_series * (std + 1e-8) + mean

    else:
        raise ValueError(f"Unknown scaler type: {scaler_info['type']}")

In [2]:
%%time
# Load scaler stats
with open("artifacts/scalers.json", "r") as f:
    stats_dict = json.load(f)

# Load scaled data
scaled_df = pd.read_parquet("data/intermediate/m3-monthly_scaled.parquet")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize an empty Series to collect results
inverted_series = pd.Series(index=scaled_df.index, dtype=float)

# Loop through each group with tqdm
for uid, group in tqdm(scaled_df.groupby("unique_id", sort=False), desc="Inverting"):
    scaled_tensor = torch.tensor(group["y_scaled"].values, dtype=torch.float32)
    inverted_tensor = invert_scaling_torch(scaled_tensor, uid, stats_dict, device=device)
    inverted_series.loc[group.index] = inverted_tensor.cpu().numpy()

# Assign back to DataFrame
scaled_df["y_inverted"] = inverted_series

Inverting:   0%|          | 0/1428 [00:00<?, ?it/s]

Inverting:   0%|          | 1/1428 [00:00<02:23,  9.92it/s]

Inverting:  42%|████▏     | 600/1428 [00:00<00:00, 3516.12it/s]

Inverting:  84%|████████▍ | 1205/1428 [00:00<00:00, 4668.81it/s]

Inverting: 100%|██████████| 1428/1428 [00:00<00:00, 4220.31it/s]

CPU times: user 290 ms, sys: 64.3 ms, total: 354 ms
Wall time: 373 ms
